In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [2]:
import json

# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

In [3]:
# define a function
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
            },
            "required": ["location"],
        },
    }
]

In [4]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Brussels?"
    }
]

In [5]:
# Call the ChatCompletion endpoint
response = openai.chat.completions.create(
model="gpt-3.5-turbo",
messages=messages,
functions=functions,
function_call="auto")

In [6]:
print(response)

ChatCompletion(id='chatcmpl-AVadHeprCPNvFo7Dwcc2gQWHQPG6b', choices=[Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=FunctionCall(arguments='{"location":"Brussels","unit":"celsius"}', name='get_current_weather'), tool_calls=None))], created=1732092347, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=21, prompt_tokens=82, total_tokens=103, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))


In [7]:
response_message = response.choices[0].message.function_call.arguments

In [8]:
print(response_message)

{"location":"Brussels","unit":"celsius"}


In [9]:
observation = get_current_weather(response_message)

In [10]:
messages.append(
        {
            "role": "function",
            "name": "get_current_weather",
            "content": observation,
        }
)

In [11]:
response = openai.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages,
)
print(response)

ChatCompletion(id='chatcmpl-AVadnEj1tZiiabWK9rHFhJWzmYjVU', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The current weather in Brussels is sunny and windy with a temperature of 22°C (72°F).', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1732092379, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=20, prompt_tokens=60, total_tokens=80, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))


In [22]:
import json

# Function to recursively serialize the ChatCompletion object
def serialize_chatcompletion(obj):
    if isinstance(obj, list):
        # Serialize each item in a list
        return [serialize_chatcompletion(item) for item in obj]
    elif hasattr(obj, "__dict__"):
        # Convert custom objects to dictionaries
        return {key: serialize_chatcompletion(value) for key, value in vars(obj).items()}
    else:
        # Return primitive types as-is
        return obj

# Assuming `response` is your ChatCompletion object
response_json = json.dumps(serialize_chatcompletion(response), indent=4)

print(response_json)


{
    "id": "chatcmpl-AUrRKP9X8Q7Nkj4pmnf2D3Ha61EZf",
    "choices": [
        {
            "finish_reason": "function_call",
            "index": 0,
            "logprobs": null,
            "message": {
                "content": null,
                "refusal": null,
                "role": "assistant",
                "audio": null,
                "function_call": {
                    "arguments": "{\"location\":\"Brussels\",\"unit\":\"celsius\"}",
                    "name": "get_current_weather"
                },
                "tool_calls": null
            }
        }
    ],
    "created": 1731918626,
    "model": "gpt-3.5-turbo-0125",
    "object": "chat.completion",
    "service_tier": null,
    "system_fingerprint": null,
    "usage": {
        "completion_tokens": 21,
        "prompt_tokens": 82,
        "total_tokens": 103,
        "completion_tokens_details": {
            "accepted_prediction_tokens": 0,
            "audio_tokens": 0,
            "reasoning_tokens":